
<div style="text-align: left;">
<table style="width:100%; background-color:transparent;">
  <tr style="background-color:transparent;">
    <td style="background-color:transparent;"><a href="http://www.datascience-paris-saclay.fr">
<img border="0" src="http://project.inria.fr/saclaycds/files/2017/02/logoUPSayPlusCDS_990.png" width="90%"> </td>
<img border="0" src="http://www.auchylesmines.fr/wp-content/uploads/2015/03/Logo-securite-routiere-651x263.jpg" width="60%"> </td>
  </tr>
</table> 
</div>

<center><h1>France accident challenge</h1></center>
<br/>
<center>Nayel Bettache (ENSAE), Axel Marchand (ENSAE), Solène Cochennec (HEC-ENSAE), Rodrigue Rillardon (ENSAE)</center>

Predicting accidents using Open Data Gouv Data: [Open Data Gouv](https://www.data.gouv.fr/fr/)

## Table of Contents

0. [Introduction](#Introduction)
1. [Data](#Data)
3. [Score metric](#Score-metric)
4. [Data exploration](#Data-exploration)
5. [Predictions](#Predictions)
6. [Record linkage](#Record-linkage)
7. [Submission structure](#Submission-structure)
8. [Local testing](#Local-testing-(before-submission))
9. [Submitting to RAMP studio](#Submitting-to-[ramp.studio](http://ramp.studio))
10. [More information](#More-information)

# Introduction

The objective of this challenge is to work with 'massive' and 'raw' data. We have a really high number of datasets, with most of the data either useless or at first not explained or use. Most real data is dirty and the availability of high-quality, open-source ML and data analysis frameworks (such as [scikit-learn](https://scikit-learn.org/),
[pandas](https://pandas.pydata.org/)...) means that the next frontier for tooling and automation lies in preprocessing. This challenge aims to investigate methodologies to perform statistical analysis directly on the original dirty data.

We have at our disposal about 14 years of data about the road accidents in France from 2004 to 2018. We will only use 3 years to avoid using a dataset too huge. 

There are twelve datasets in this challenge, describing the years 2018, 2017, 2016. Each year has 4 differents datasets

* `caracteristiques_{YEAR}.csv` - basic data about the accidents.
* `lieux_{YEAR}.csv` - data about the location of the accidents.
* `usagers_{YEAR}.csv` - data about the people involved in the accidents.
* `vehicules_{YEAR}.csv` - data about the vehicles involved.


# Aim

The predictive aim of this challenge is to use the different datasets to predict the number of accidents for each location in a day. It is advised that you use all datasets, as it improves the prediction (see [Score comparison](#Score-comparison)), but using only the `caracteristiques_{YEAR}.csv` dataset is also allowed.

In [1]:
!python download_data.py

=> File saved as data/vehicules_2018.csv
=> File saved as data/usagers_2018.csv
=> File saved as data/lieux_2018.csv
=> File saved as data/caracteristiques_2018.csv
=> File saved as data/vehicules_2017.csv
=> File saved as data/usagers_2017.csv
=> File saved as data/lieux_2017.csv
=> File saved as data/caracteristiques_2017.csv
=> File saved as data/vehicules_2016.csv
=> File saved as data/usagers_2016.csv
=> File saved as data/lieux_2016.csv
=> File saved as data/caracteristiques_2016.csv


# Data

## Caracteristiques

## Lieux

## Usagers

## Vehicles

## Training and test

# Score metric

# Data exploration

## Caracteristiques

## Lieux

## Usagers

TEST